In [1]:
# pip install pydantic

In [2]:
# pip install boto3

In [3]:
# pip install gspread

In [4]:
# pip install openai

In [5]:
# pip install pikepdf

In [6]:
# pip install pypdf

In [7]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work


In [8]:
from typing import Any, Dict, List

import boto3
import yaml

from core.task_processor import TaskProcessor
from core.task_publisher import TaskPublisher

In [9]:
_sqs = None
_processor = None
_queue_url = None

In [10]:
with open("../config/tenants/dev.yaml", "r", encoding="utf-8") as f:
    tenant_config: Dict[str, Any] = yaml.safe_load(f)

dynamodb_config = tenant_config.get("dynamodb", {}) or {}
dynamodb = boto3.resource("dynamodb", region_name=dynamodb_config.get("region"))

messages_table = dynamodb.Table(dynamodb_config.get("messages_table"))
processes_table = dynamodb.Table(dynamodb_config.get("processes_table"))
contacts_table = dynamodb.Table(dynamodb_config.get("contacts_table"))
tasks_table = dynamodb.Table(dynamodb_config.get("tasks_table"))

sqs_config = tenant_config.get("sqs", {}) or {}
_sqs = boto3.client("sqs", region_name=sqs_config.get("region"))

_queue_url = (sqs_config.get("tasks_url"))

s3_config = tenant_config.get("s3", {}) or {}
_s3 = boto3.client("s3", region_name=s3_config.get("region"))

_processor = TaskProcessor(
    tenant_config=tenant_config,
    messages_table=messages_table,
    processes_table=processes_table,
    contacts_table=contacts_table,
    tasks_table=tasks_table,
    s3_client=_s3,
    task_publisher=TaskPublisher(_sqs, _queue_url),
)

print("INIT: processor ready")

INIT: processor ready


In [11]:
import json

body = {
    "task_id": "5bbcac21-2179-4df3-a4c8-10f75f23e6cf",
    "task_type": "PDF_UNLOCK",
    "agent_type": "ACCOUNTING_ASSISTANT",
    "process_type": "WHATSAPP_DOCUMENT_PIPELINE",
    "context_key": {
        "msg_id": "wamid.HBgNNTIxNTU3MTk2OTg0OBUCABIYFDJBQUY4RDA5QTJDRjhEOEY5NzE3AA==",
        "document_id": "25749488007978009"
    },
    "payload": {
        "identity": "whatsapp:525571969848",
        "phone": "525571969848",
        "msg_id": "wamid.HBgNNTIxNTU3MTk2OTg0OBUCABIYFDJBQUY4RDA5QTJDRjhEOEY5NzE3AA==",
        "document_id": "25749488007978009",
        "timestamp_iso": "2025-12-21T05:04:50",
        "timestamp_epoch": 1766296847,
        "document": {
            "media_id": "25749488007978009",
            "media_type": "document",
            "mime_type": "application/pdf",
            "filename": "CSF - FITLOG 01-09-2025 1.pdf"
        }
    },
    "timestamp_iso": "2025-12-21T05:45:58.713494+00:00",
    "timestamp_epoch": 1766295958
}
task = json.dumps(body)

In [12]:
try:
    message_id = "Fake"
    processed, remaining = _processor.process(task)
    print(f"PROCESS: messageId={message_id} processed={processed} remaining={remaining}")

    if remaining < 0:
        print(f"DROP: messageId={message_id} stale")

    delay = max(0, min(int(remaining), 43200))
    print(f"DEFER: messageId={message_id} delay={delay}s")

except Exception as e:
    print(f"ERROR: messageId={message_id} err={repr(e)}")

1111
{'identity': 'whatsapp:525571969848', 'contact_id': 'c_88fe92bfdc65', 'password': None}
'NoneType' object is not subscriptable
PROCESS: messageId=Fake processed=True remaining=0
DEFER: messageId=Fake delay=0s
